In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.8 MB/s eta 0:00:00


In [3]:
import os
import gc
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
import re
import html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam, SGD, AdamW, RAdam
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import StratifiedKFold,StratifiedGroupKFold,GroupKFold
from sklearn.metrics import log_loss,f1_score, recall_score, accuracy_score, precision_score

#from transformers import AutoModel, AutoConfig, AutoTokenizer, AdamW, DataCollatorWithPadding
#from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
import lightgbm as lgb
import catboost as ctb
from catboost import *
from sklearn import svm
from sklearn.svm import LinearSVR
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
DIR = "/content/drive/MyDrive/Competitions/probspace/研究論文の国際学会採択予測"
INPUT_DIR = os.path.join(DIR,"input")
OUTPUT_DIR = os.path.join(DIR,"output")
CUSTOM_MODEL_DIR = os.path.join(OUTPUT_DIR,'clrp_deberta_v3_base')
OUTPUT_MODEL_DIR = DIR + '/output/EXP035/'
OUTPUT_EXP_DIR = OUTPUT_MODEL_DIR + '/ABSTRACT/'
if not os.path.exists(OUTPUT_MODEL_DIR):
    os.makedirs(OUTPUT_MODEL_DIR)
if not os.path.exists(OUTPUT_EXP_DIR):
    os.makedirs(OUTPUT_EXP_DIR)

In [60]:
def get_score(labels, outputs):
    thresh = 0.5
    y_pred = outputs
    y_true = labels
    f_score = f1_score(y_true, (y_pred>thresh).astype(int))
    r_score = recall_score(y_true, (y_pred>thresh).astype(int))
    p_score = precision_score(y_true, (y_pred>thresh).astype(int))
    print(f"f1 score : {f_score}")
    print(f"recall score : {r_score}")
    print(f"precision score : {p_score}")
    print(accuracy_score(y_true, (y_pred>thresh).astype(int)))

def get_acc_score(labels, outputs):
    y_pred = outputs
    y_true = labels
    best_score = 0
    best_thresh = 0.5
    for thresh in np.arange(0.1, 0.80, 0.01):
        thresh = np.round(thresh, 2)
        score = accuracy_score(y_true, (y_pred>thresh).astype(int))
        #print("Accuracy score at threshold {0} is {1}".format(thresh, score))
        if score > best_score:
          best_score = score
          best_thresh = thresh
    print(f"thresh : {best_thresh}")
    print(accuracy_score(y_true, (y_pred>best_thresh).astype(int)))
    return best_thresh

In [5]:
oof_abst = pd.read_pickle(os.path.join(OUTPUT_EXP_DIR,"oof_abstract_df.pkl"))
oof_title = pd.read_pickle(os.path.join(OUTPUT_MODEL_DIR,"oof_title_df.pkl"))

oof_abst.rename(columns={"pred":"abst_pred"},inplace=True)
oof_title.rename(columns={"pred":"title_pred"},inplace=True)

oof = oof_abst.merge(oof_title[["id","title_pred"]],how="left",on="id")
display(oof)

,id,title,year,abstract,keywords,y,texts,kfold,abst_pred,title_pred
0,2,Learning to Compute Word Embeddings On the Fly,2018,Words in natural language follow a Zipfian dis...,"NLU, word embeddings, representation learning",0,Words in natural language follow a Zipfian dis...,0,0.263803,0.579783
1,7,Automatic Parameter Tying in Neural Networks,2018,"Recently, there has been growing interest in m...","neural network, quantization, compression",0,"Recently, there has been growing interest in m...",0,0.354146,0.194521
2,8,GraphGAN: Generating Graphs via Random Walks,2018,We propose GraphGAN - the first implicit gener...,"GAN, graphs, random walks, implicit generative...",0,We propose GraphGAN - the first implicit gener...,0,0.388363,0.318945
3,26,Optimizing the Latent Space of Generative Netw...,2018,Generative Adversarial Networks (GANs) have ac...,"generative models, latent variable models, ima...",0,Generative Adversarial Networks (GANs) have ac...,0,0.438576,0.385699
4,37,Neural Task Graph Execution,2018,In order to develop a scalable multi-task rein...,"deep reinforcement learning, task execution, i...",0,In order to develop a scalable multi-task rein...,0,0.239491,0.264982
...,...,...,...,...,...,...,...,...,...,...
4969,4923,Reconstructing continuous distributions of 3D ...,2020,Cryo-electron microscopy (cryo-EM) is a powerf...,"generative models, proteins, 3D reconstruction...",1,Cryo-electron microscopy (cryo-EM) is a powerf...,9,0.568993,0.324022
4970,4929,GenDICE: Generalized Offline Estimation of Sta...,2020,An important problem that arises in reinforcem...,"Off-policy Policy Evaluation, Reinforcement Le...",1,An important problem that arises in reinforcem...,9,0.649682,0.373118
4971,4950,Geometric Analysis of Nonconvex Optimization L...,2020,Learning overcomplete representations finds ma...,"dictionary learning, sparse representations, n...",1,Learning overcomplete representations finds ma...,9,0.454165,0.339360
4972,4964,Restricting the Flow: Information Bottlenecks ...,2020,Attribution methods provide insights into the ...,"Attribution, Informational Bottleneck, Interpr...",1,Attribution methods provide insights into the ...,9,0.509551,0.344005


In [7]:
abst_thresh = [0.67, 0.54, 0.59, 0.59, 0.79, 0.49, 0.76, 0.55, 0.46, 0.67]
title_thresh = [0.4, 0.56, 0.45, 0.77, 0.41, 0.65, 0.31, 0.68, 0.46, 0.43]

In [9]:
oof_df = pd.DataFrame()
for i in range(10):
  _df = oof.loc[oof.kfold==i]
  _df["title_thresh"] = title_thresh[i]
  _df["abst_thresh"] = abst_thresh[i]
  oof_df = pd.concat([oof_df,_df])
oof_df = oof_df.sort_index()
display(oof_df)

,id,title,year,abstract,keywords,y,texts,kfold,abst_pred,title_pred,title_thresh,abst_thresh
0,2,Learning to Compute Word Embeddings On the Fly,2018,Words in natural language follow a Zipfian dis...,"NLU, word embeddings, representation learning",0,Words in natural language follow a Zipfian dis...,0,0.263803,0.579783,0.40,0.67
1,7,Automatic Parameter Tying in Neural Networks,2018,"Recently, there has been growing interest in m...","neural network, quantization, compression",0,"Recently, there has been growing interest in m...",0,0.354146,0.194521,0.40,0.67
2,8,GraphGAN: Generating Graphs via Random Walks,2018,We propose GraphGAN - the first implicit gener...,"GAN, graphs, random walks, implicit generative...",0,We propose GraphGAN - the first implicit gener...,0,0.388363,0.318945,0.40,0.67
3,26,Optimizing the Latent Space of Generative Netw...,2018,Generative Adversarial Networks (GANs) have ac...,"generative models, latent variable models, ima...",0,Generative Adversarial Networks (GANs) have ac...,0,0.438576,0.385699,0.40,0.67
4,37,Neural Task Graph Execution,2018,In order to develop a scalable multi-task rein...,"deep reinforcement learning, task execution, i...",0,In order to develop a scalable multi-task rein...,0,0.239491,0.264982,0.40,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...
4969,4923,Reconstructing continuous distributions of 3D ...,2020,Cryo-electron microscopy (cryo-EM) is a powerf...,"generative models, proteins, 3D reconstruction...",1,Cryo-electron microscopy (cryo-EM) is a powerf...,9,0.568993,0.324022,0.43,0.67
4970,4929,GenDICE: Generalized Offline Estimation of Sta...,2020,An important problem that arises in reinforcem...,"Off-policy Policy Evaluation, Reinforcement Le...",1,An important problem that arises in reinforcem...,9,0.649682,0.373118,0.43,0.67
4971,4950,Geometric Analysis of Nonconvex Optimization L...,2020,Learning overcomplete representations finds ma...,"dictionary learning, sparse representations, n...",1,Learning overcomplete representations finds ma...,9,0.454165,0.339360,0.43,0.67
4972,4964,Restricting the Flow: Information Bottlenecks ...,2020,Attribution methods provide insights into the ...,"Attribution, Informational Bottleneck, Interpr...",1,Attribution methods provide insights into the ...,9,0.509551,0.344005,0.43,0.67


In [11]:
skf = StratifiedKFold(n_splits=8,shuffle=True,random_state=18)
for fold, ( _, val_) in enumerate(skf.split(oof_df, oof_df.y)):
    oof_df.loc[val_ , "fold"] = int(fold)
    
oof_df["fold"] = oof_df["fold"].astype(int)
display(oof_df.groupby('fold').size())

fold
0    622
1    622
2    622
3    622
4    622
5    622
6    621
7    621
dtype: int64

In [28]:
lgbm_params = {
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': 0.05,
        'boosting': "gbdt",
        'seed': 42,
        "reg_alpha":0.1,
        #'reg_lambda': 0.5,
        "n_estimators":3000,
        'min_child_weight': 1,
        'colsample_bytree': 0.4
    }

def get_oof_LGBM_pred(df):
  oof_df = pd.DataFrame()
  #test_preds = np.zeros((len(test), 8))


  for fold in range(8):
    print("="*15,f" OUTER CV FOLD {fold} ","="*15)

    train_folds = df.loc[df.fold!=fold].reset_index(drop=True)
    valid_folds = df.loc[df.fold==fold].reset_index(drop=True)



    train_X = train_folds[features].values
    valid_X = valid_folds[features].values
    #test_X = test[features].values
    train_labels = train_folds[target].values
    valid_labels = valid_folds[target].values

    clf = lgb.LGBMRegressor(**lgbm_params)
    clf.fit(train_X, train_labels,
            eval_set=[(valid_X, valid_labels)],
            verbose = 1000,
            early_stopping_rounds=50)
      
    valid_folds[f"lgbm_pred"] = clf.predict(valid_X)
    #test_preds[:, fold] = clf.predict(test_X)
    oof_df = pd.concat([oof_df, valid_folds])
    
    get_acc_score(valid_labels, valid_folds["lgbm_pred"])

  get_acc_score(oof_df[target], oof_df[f"lgbm_pred"])
  return oof_df

In [74]:
features = ["abst_pred",	"title_pred",	"title_thresh",	"abst_thresh"]
target = "y"

In [30]:
get_oof_LGBM_pred(oof_df)

===============  OUTER CV FOLD 0  ===============
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
thresh : 0.43
0.7106109324758842
===============  OUTER CV FOLD 1  ===============
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
thresh : 0.53
0.707395498392283
===============  OUTER CV FOLD 2  ===============
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
thresh : 0.47
0.7170418006430869
===============  OUTER CV FOLD 3  ===============
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
thresh : 0.53
0.7138263665594855
===============  OUTER CV FOLD 4  ===============
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
thresh : 0.45
0.6977491961414791
===============  OUTER CV FOLD 5  ============

,id,title,year,abstract,keywords,y,texts,kfold,abst_pred,title_pred,title_thresh,abst_thresh,fold,lgbm_pred
0,63,Revisiting Knowledge Base Embedding as Tensor ...,2018,We study the problem of knowledge base (KB) em...,Knowledge base embedding,0,We study the problem of knowledge base (KB) em...,0,0.213330,0.119408,0.40,0.67,0,0.209388
1,95,Convolutional Normalizing Flows,2018,Bayesian posterior inference is prevalent in v...,NaN,0,Bayesian posterior inference is prevalent in v...,0,0.053578,0.277237,0.40,0.67,0,0.188898
2,164,"Universality, Robustness, and Detectability of...",2018,Classifiers such as deep neural networks have ...,"adversarial examples, adversarial training, un...",0,Classifiers such as deep neural networks have ...,0,0.239694,0.323207,0.40,0.67,0,0.295667
3,245,Trace norm regularization and faster inference...,2018,We propose and evaluate new techniques for com...,"LVCSR, speech recognition, embedded, low rank ...",0,We propose and evaluate new techniques for com...,0,0.471859,0.136238,0.40,0.67,0,0.393904
4,248,How do deep convolutional neural networks lear...,2018,Prior work on speech and audio processing has ...,"Convolutional neural networks, Audio processin...",0,Prior work on speech and audio processing has ...,0,0.123201,0.150433,0.40,0.67,0,0.222355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,4826,Defending Against Physically Realizable Attack...,2020,We study the problem of defending deep neural ...,"defense against physical attacks, adversarial ...",1,We study the problem of defending deep neural ...,9,0.560774,0.333068,0.43,0.67,7,0.385323
617,4851,Gradientless Descent: High-Dimensional Zeroth-...,2020,Zeroth-order optimization is the process of mi...,Zeroth Order Optimization,1,Zeroth-order optimization is the process of mi...,9,0.696941,0.313088,0.43,0.67,7,0.559716
618,4860,The Logical Expressiveness of Graph Neural Net...,2020,The ability of graph neural networks (GNNs) fo...,"Graph Neural Networks, First Order Logic, Expr...",1,The ability of graph neural networks (GNNs) fo...,9,0.605632,0.434889,0.43,0.67,7,0.453353
619,4883,Understanding Why Neural Networks Generalize W...,2020,As deep neural networks (DNNs) achieve tremend...,"DNN, generalization, GSNR, gradient descent",1,As deep neural networks (DNNs) achieve tremend...,9,0.292124,0.367208,0.43,0.67,7,0.228385


In [75]:
def get_oof_linear_pred(df, model, model_name=""):
  oof_df = pd.DataFrame()

  for fold in range(8):
    print("="*15,f" OUTER CV FOLD {fold} ","="*15)

    train_folds = df.loc[df.fold!=fold].reset_index(drop=True)
    valid_folds = df.loc[df.fold==fold].reset_index(drop=True)

    sc = StandardScaler()
    train_X = train_folds[features].values
    train_X =  sc.fit_transform(train_X)
    valid_X = valid_folds[features].values
    valid_X = sc.transform(valid_X)
    #test_X = test[features].values
    train_labels = train_folds[target].values
    valid_labels = valid_folds[target].values

    clf = model
    clf.fit(train_X, train_labels)
      
    valid_folds[f"{model_name}_pred"] = clf.predict_proba(valid_X)[:,1]
    #test_preds[:, fold] = clf.predict(test_X)
    thresh = get_acc_score(valid_labels, valid_folds[f"{model_name}_pred"])
    valid_folds["thresh"] = thresh
    valid_folds["pred"] = valid_folds[f"{model_name}_pred"].apply(lambda x : 1 if x > thresh else 0)
    oof_df = pd.concat([oof_df, valid_folds])

  thresh = get_score(oof_df[target], oof_df[f"pred"])
  return oof_df

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

logit = LogisticRegression()

In [77]:
logit_df = get_oof_linear_pred(oof_df,logit,"logit")
logit_df

===============  OUTER CV FOLD 0  ===============
thresh : 0.61
0.7106109324758842
===============  OUTER CV FOLD 1  ===============
thresh : 0.48
0.702572347266881
===============  OUTER CV FOLD 2  ===============
thresh : 0.51
0.7234726688102894
===============  OUTER CV FOLD 3  ===============
thresh : 0.55
0.7170418006430869
===============  OUTER CV FOLD 4  ===============
thresh : 0.47
0.707395498392283
===============  OUTER CV FOLD 5  ===============
thresh : 0.5
0.7347266881028939
===============  OUTER CV FOLD 6  ===============
thresh : 0.43
0.7085346215780999
===============  OUTER CV FOLD 7  ===============
thresh : 0.47
0.7230273752012882
f1 score : 0.2831050228310502
recall score : 0.18319107025607353
precision score : 0.6227678571428571
0.7159227985524729


,id,title,year,abstract,keywords,y,texts,kfold,abst_pred,title_pred,title_thresh,abst_thresh,fold,logit_pred,thresh,pred
0,63,Revisiting Knowledge Base Embedding as Tensor ...,2018,We study the problem of knowledge base (KB) em...,Knowledge base embedding,0,We study the problem of knowledge base (KB) em...,0,0.213330,0.119408,0.40,0.67,0,0.193549,0.61,0
1,95,Convolutional Normalizing Flows,2018,Bayesian posterior inference is prevalent in v...,NaN,0,Bayesian posterior inference is prevalent in v...,0,0.053578,0.277237,0.40,0.67,0,0.151234,0.61,0
2,164,"Universality, Robustness, and Detectability of...",2018,Classifiers such as deep neural networks have ...,"adversarial examples, adversarial training, un...",0,Classifiers such as deep neural networks have ...,0,0.239694,0.323207,0.40,0.67,0,0.238807,0.61,0
3,245,Trace norm regularization and faster inference...,2018,We propose and evaluate new techniques for com...,"LVCSR, speech recognition, embedded, low rank ...",0,We propose and evaluate new techniques for com...,0,0.471859,0.136238,0.40,0.67,0,0.334920,0.61,0
4,248,How do deep convolutional neural networks lear...,2018,Prior work on speech and audio processing has ...,"Convolutional neural networks, Audio processin...",0,Prior work on speech and audio processing has ...,0,0.123201,0.150433,0.40,0.67,0,0.161060,0.61,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,4826,Defending Against Physically Realizable Attack...,2020,We study the problem of defending deep neural ...,"defense against physical attacks, adversarial ...",1,We study the problem of defending deep neural ...,9,0.560774,0.333068,0.43,0.67,7,0.436993,0.47,0
617,4851,Gradientless Descent: High-Dimensional Zeroth-...,2020,Zeroth-order optimization is the process of mi...,Zeroth Order Optimization,1,Zeroth-order optimization is the process of mi...,9,0.696941,0.313088,0.43,0.67,7,0.525441,0.47,1
618,4860,The Logical Expressiveness of Graph Neural Net...,2020,The ability of graph neural networks (GNNs) fo...,"Graph Neural Networks, First Order Logic, Expr...",1,The ability of graph neural networks (GNNs) fo...,9,0.605632,0.434889,0.43,0.67,7,0.490951,0.47,1
619,4883,Understanding Why Neural Networks Generalize W...,2020,As deep neural networks (DNNs) achieve tremend...,"DNN, generalization, GSNR, gradient descent",1,As deep neural networks (DNNs) achieve tremend...,9,0.292124,0.367208,0.43,0.67,7,0.277058,0.47,0
